In [1]:
import numpy as np
import pandas as pd
import feather
from pandasql import sqldf
from fisher import pvalue
from scipy import stats
from pprint import pprint
from collections import Counter

In [32]:
p = 0.3
n_cnv = 40
n_max = 10
const = 1
# trun_sum = sum([p*(1-p)**(n_max-1-i) for i in range(n_max)])
# seed = 999
# np.random.seed(seed)
cnv_len = np.random.choice((np.random.geometric(p, size=n_cnv) - 1), n_cnv, replace=False)
cnv_len = cnv_len[cnv_len <= 10].tolist()
print (cnv_len)

array([ 0,  0,  2,  0,  1,  3,  8,  0,  4,  2,  1,  0,  2,  1,  1,  1,  0,
        2,  3,  1,  4,  0,  0,  3,  2,  5,  0,  8,  2,  4,  0,  0, 10,  6,
        0,  2,  0,  1,  2,  0])
[0,
 0,
 2,
 0,
 1,
 3,
 8,
 0,
 4,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 3,
 1,
 4,
 0,
 0,
 3,
 2,
 5,
 0,
 8,
 2,
 4,
 0,
 0,
 10,
 6,
 0,
 2,
 0,
 1,
 2,
 0]


In [33]:
print (cnv_len)

[0, 0, 2, 0, 1, 3, 8, 0, 4, 2, 1, 0, 2, 1, 1, 1, 0, 2, 3, 1, 4, 0, 0, 3, 2, 5, 0, 8, 2, 4, 0, 0, 10, 6, 0, 2, 0, 1, 2, 0]


In [40]:
[np.random.choice(range(n_max+1-i)) if i!=0 else -1 for i in cnv_len]
    

[-1,
 -1,
 6,
 -1,
 7,
 1,
 1,
 -1,
 5,
 7,
 7,
 -1,
 1,
 5,
 7,
 6,
 -1,
 6,
 1,
 2,
 0,
 -1,
 -1,
 5,
 8,
 1,
 -1,
 0,
 8,
 3,
 -1,
 -1,
 0,
 2,
 -1,
 1,
 -1,
 1,
 8,
 -1]